In [ ]:
# Video (Fourier transform): https://www.youtube.com/watch?v=rzCO5fQysw0 -- AWESOME APPLICATIONS

def read_file(file_name):
    file = open(file_name)
    data = file.read()
    file.close()
    return data

data_as_string = read_file('data/sea_level.txt')
len(data_as_string)

In [ ]:
data_as_list = data_as_string.split("\n")

In [ ]:
good_lines = data_as_list[52:-1]

In [ ]:
good_lines[0]

In [ ]:
# python's str.split() works for most whitespaces including tabs newlines and spaces

good_lines[0].split()

In [ ]:
split_data = [line.split() for line in good_lines]

# I used a 'list comprehension' [f(x) for x in list]
# e.g: [x**2 for x in range(5)] evaluates to [0, 1, 4, 9, 16]
# similar to list(map(lambda x: x**2, range(5)))

In [ ]:
split_data[-2:]

In [ ]:
# converting all data to float

split_data_float = [[float(item) for item in row] for row in split_data]

In [ ]:
split_data_float[-2:]

In [ ]:
data_as_list[31:46]

In [ ]:
#   [[row1],[row2],[row3],..]

# Currently, each row is a single list. But we want each column as a single list.

#   [[col1],[col2],[col3],..]

# row1,           c c c
# row2,    zip    o o o
# row3,  <----->  l l l 
# row4            1,2,3

_, _, year, _, _, GMSL_1, _, _, _, _, _, _ = zip(*split_data_float)
# & is the "splat operator"

# another way:
# from operator import itemgetter as item
# year, GMSL_1 = item(2,5)(list(zip(*split_data_float)))

In [ ]:
year[-10:]

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

# Uncomment following line to get sharper figures on mac (might work on windows as well)
#%config InlineBackend.figure_formats = ["retina"]

In [ ]:
plt.plot(year,GMSL_1)
plt.title("GMSL vs time")
plt.xlabel("Year")
plt.xlabel("GMSL")

In [ ]:
import numpy as np

# quick linear regression
a, b = np.polyfit(year, GMSL_1, 1)
GMSL_1_trend = [a*x+b for x in year]
############    ^^^ Use list comprehension

plt.plot(year,GMSL_1)
plt.plot(year,GMSL_1_trend)
plt.title("GMSL and linear fit")
plt.xlabel("Year")
plt.ylabel("GMSL")

In [ ]:
# subtracting the trend using a list comprehension

GMSL_1_sub_trend = [G-T for G,T in zip(GMSL_1,GMSL_1_trend)]
plt.plot(year,GMSL_1_sub_trend)
plt.title("GMSL - trend")
plt.xlabel("Year")
plt.ylabel("GMSL")

In [ ]:
# checking if data points are uniformly distributed in time
# If the standard deviation was too high compared to mean, 
# I would interpolate data on a uniform time axis first
#  e.g. on np.linspace(1993.5,2019.1,900)

diff_years = np.diff(year)
# ^^^ The first difference is given by out[n] = a[n+1] - a[n] along the given axis, 
# higher differences are calculated by using diff recursively.
avg_diff_years = np.mean(diff_years)
std_diff_years = np.std(diff_years)
avg_diff_years, std_diff_years

In [ ]:
fft_GMSL_1 = np.absolute(np.fft.fft(GMSL_1_sub_trend))[:-850]

## fft_GMSL_1 = np.absolute(np.fft.fft(GMSL_1_sub_trend))[:450]

dt_mean = avg_diff_years
sampling_freq = 1/dt_mean # this has units 1/year (per year)
########################### ^^^ Always good to think about units
signal_len = len(GMSL_1_sub_trend)

freq = [sampling_freq*partial_len/signal_len for partial_len in range(len(fft_GMSL_1))]
plt.plot(freq,fft_GMSL_1)
plt.plot([1/3]*2,[0,2200],'r--')
plt.title("Spectrum of GMSL data")
plt.xlabel("Cycles per year")
plt.ylabel("Power")
print("len(fft_data =", len(fft_GMSL_1))

In [ ]:
# Smoothing data: https://stackoverflow.com/questions/20618804/how-to-smooth-a-curve-in-the-right-way
x = np.linspace(0,2*np.pi,100)
y = np.sin(x) + np.random.random(100) * 0.8

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

# Reference: https://docs.scipy.org/doc/numpy/reference/generated/numpy.convolve.html
#            http://docs.astropy.org/en/stable/convolution/using.html
#            https://www.youtube.com/watch?v=zZS6DYkyZIY (VIDEO)
#            https://www.youtube.com/watch?v=rzCO5fQysw0 (EXTREMELY COOL VIDEO!)
#            https://www.youtube.com/watch?v=FjmwwDHT98c (VIDEO)

plt.plot(x, y,'o')
plt.plot(x, smooth(y,3), 'r-', lw=2)
plt.plot(x, smooth(y,19), 'g-', lw=2)

In [ ]:
box_pts = 5
box = np.ones(box_pts)/box_pts
box

In [ ]:
smoothing_window = 10
fft_GMSL_1_smooth = np.absolute(np.fft.fft(smooth(GMSL_1_sub_trend,smoothing_window)))[:-850]

In [ ]:
len(fft_GMSL_1), len(fft_GMSL_1_smooth)

In [ ]:
plt.plot(freq,fft_GMSL_1_smooth)
# plt.plot(freq,fft_GMSL_1)
plt.title("Spectrum of smoothed GMSL data")
plt.xlabel("Cycles per year")
plt.ylabel("Power")

In [ ]:
fft_GMSL_1_smooth

In [ ]:
fft_GMSL_1_smooth[1], fft_GMSL_1_smooth[9], fft_GMSL_1_smooth[26], fft_GMSL_1_smooth[52],